In [1]:
import numpy as np
import time
from math import sin,cos,exp,log,pi
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
from numba import njit
import threading

In [2]:
C1=1.2
C2=0.3
C3=3
C4=pi+1
n=1_000_000



In [3]:

problem = {
    'num_vars' : 5,
    'names' : ['x1', 'x2', 'x3', 'x4','x5'],
    'bounds' : [[-pi,pi],
                [-pi,pi],
                [-pi,pi],
                [-pi,pi],
                [-pi,pi]
               ]
}

## Baseline 

In [8]:
def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])

start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = func(X)
print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 6.090597867965698 seconds
model evaluation took 3.4115967750549316 seconds
SA took 21.383636951446533 seconds


In [9]:
S

{'S1': array([7.21975662e-05, 3.02326722e-06, 1.65221450e-05, 8.96472376e-01,
        1.04785599e-04]),
 'S1_conf': array([1.71408004e-04, 9.03772073e-06, 2.72455059e-05, 8.99996808e-01,
        1.35701837e-04]),
 'ST': array([2.23019123e-01, 1.48633457e-06, 1.38171038e-05, 9.57104461e-01,
        8.10596148e-05]),
 'ST_conf': array([2.10184979e-01, 2.34142950e-06, 2.17142745e-05, 2.56187431e-01,
        1.27208895e-04]),
 'S2': array([[            nan, -1.12400062e-05, -1.06003548e-05,
          4.83538128e-02, -1.47413893e-05],
        [            nan,             nan,  2.83545818e-06,
         -9.12405056e-07,  2.81090891e-06],
        [            nan,             nan,             nan,
         -3.84391184e-06,  3.07989784e-05],
        [            nan,             nan,             nan,
                     nan, -4.85786777e-01],
        [            nan,             nan,             nan,
                     nan,             nan]]),
 'S2_conf': array([[           nan, 1.79181852

## Numba 

In [7]:
@njit(fastmath=True)
def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])


start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = func(X)
print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 56.46754193305969 seconds
model evaluation took 8.255743980407715 seconds
SA took 307.17661809921265 seconds


### Видно, что evalution заняло в 4-5 раз меньше чем без Numba
#### Попробуем ради интереса еще и распараллелить(результат не гарантируется)

In [8]:
@njit(fastmath=True,parallel=True)
def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])


start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = func(X)
print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 56.85862874984741 seconds


/Users/workhard/opt/anaconda3/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-8-aa2b5931d905>", line 2:
@njit(fastmath=True,parallel=True)
def func(X):
^

  self.func_ir.loc))


model evaluation took 8.097552299499512 seconds
SA took 309.0211362838745 seconds


## Multithreads

In [6]:
n=1_000_00
nThreads=8

def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])

def funcParallel(tid):
    lenParams=len(param_values)
    lenPart=int(lenParams/nThreads)
    start=tid*lenPart
    end=(tid+1)*lenPart
    #print(str(tid),str(lenParams),str(tid*lenPart),str((tid+1)*lenPart)+'\n')
    #print(tid,start,end,'\n')
    for i, X in enumerate(param_values[start:end]):
        Y[i+start] = func(X)

start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
threads=[]
for tid in range(nThreads):
    th=threading.Thread(target=funcParallel,args=(tid,))
    threads.append(th)
    th.start()
    
for th in threads:
    th.join()
    #print(th)
            
print("parallel model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 6.088142156600952 seconds
parallel model evaluation took 3.5214080810546875 seconds
SA took 20.18432307243347 seconds


#### К сожалению, мультитрединг не дал прироста скорости(прирост на уровне погрешности)

In [7]:
S

{'S1': array([7.21975662e-05, 3.02326722e-06, 1.65221450e-05, 8.96472376e-01,
        1.04785599e-04]),
 'S1_conf': array([1.19086823e-04, 8.33959392e-06, 3.47284977e-05, 8.91347907e-01,
        1.64472590e-04]),
 'ST': array([2.23019123e-01, 1.48633457e-06, 1.38171038e-05, 9.57104461e-01,
        8.10596148e-05]),
 'ST_conf': array([1.79275763e-01, 2.70436651e-06, 2.52665693e-05, 2.69256035e-01,
        1.47992083e-04]),
 'S2': array([[            nan, -1.12400062e-05, -1.06003548e-05,
          4.83538128e-02, -1.47413893e-05],
        [            nan,             nan,  2.83545818e-06,
         -9.12405056e-07,  2.81090891e-06],
        [            nan,             nan,             nan,
         -3.84391184e-06,  3.07989784e-05],
        [            nan,             nan,             nan,
                     nan, -4.85786777e-01],
        [            nan,             nan,             nan,
                     nan,             nan]]),
 'S2_conf': array([[           nan, 1.49260331